### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import librosa
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from algorithms import encode_columns, svm_model, accuracy_calculator, random_forest_model, array_column_spread
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

### Loading the dataset using Pandas
The data is found in development.csv (development set): a comma-separated values file containing the records from the development set. This portion does have the action and object columns, which you should use to obtain the labels to train and validate your models.
The dataset consists in a collection of audio file in a WAV format. 

Each record is characterized by several attributes. The following is a short description for each of them.
- path: the path of the audio file.
- speakerId: the id of the speaker.
- action: the type of action required through the intent.
- object: the device involved by intent.
- Self-reported fluency level: the speaking fluency of the speaker.
- First Language spoken: the first language spoken by the speaker.
- Current language used for work/school: the main language spoken by the speaker during daily activities.
- gender: the gender of the speaker.
- ageRange: the age range of the speaker.

In [ ]:
df = pd.read_csv("dsl_data/development.csv")
df_eval = pd.read_csv("dsl_data/evaluation.csv")

In [ ]:
y = df.values[:,3:5].sum(axis=1)
y

In [ ]:
column_names = ["Self-reported fluency level ","First Language spoken", "Current language used for work/school", "speakerId", "gender","ageRange"]


In [ ]:
encode_columns(df_eval, column_names)

In [ ]:

encode_columns(df,column_names)

In [ ]:
def audio_feature_extraction(df):
        data_array= []
        rate_array = []
        for audio in df['path']:
                data, rate = librosa.load(audio)
                data_array.append(data)
                rate_array.append(rate)

        df['data'] = data_array
        df['rate'] = rate_array

In [ ]:
audio_feature_extraction(df)
audio_feature_extraction(df_eval)

### Chroma feature

In [ ]:
def chroma_feature(df):
    chroma_array = []
    for data, rate in zip(df['data'], df['rate']):
        chroma = librosa.feature.chroma_stft(y=data, sr=rate)
        chroma_mean = np.mean(chroma, axis=1)
        chroma_array.append(chroma_mean)

    df['chroma'] = chroma_array

In [ ]:
chroma_feature(df)
chroma_feature(df_eval)


In [ ]:
# df['mean_chroma'] = array_column_spread(df,'chroma')

array_column_spread(df,'chroma')
array_column_spread(df_eval,'chroma')

### Tonnetz feature

In [ ]:
def tonnetz_feature(df):
    tonnetz_array = []
    for data, rate in zip(df['data'], df['rate']):
        tonnetz = librosa.feature.tonnetz(y=data, sr=rate)
        tonnetz_mean = np.mean(tonnetz, axis=1)
        tonnetz_array.append(tonnetz_mean)

    df['tonnetz'] = tonnetz_array

In [ ]:
tonnetz_feature(df)
tonnetz_feature(df_eval)

In [ ]:
# df['mean_tonnetz'] = array_column_spread(df,'tonnetz')
array_column_spread(df, 'tonnetz')
array_column_spread(df_eval, 'tonnetz')

### Spectral Contrast

In [ ]:
def spectral_contrast(df):
    spectral_contrast_array = []
    for data, rate in zip(df['data'], df['rate']):
        spectral_contrast = librosa.feature.spectral_contrast(y=data, sr=rate)
        spectral_contrast_mean = np.mean(spectral_contrast, axis=1)
        spectral_contrast_array.append(spectral_contrast_mean)
    df['spectral_contrast'] = spectral_contrast_array    

In [ ]:
spectral_contrast(df)
spectral_contrast(df_eval)


In [ ]:
# df['mean_spectral_contrast'] = array_column_spread(df,'spectral_contrast')
array_column_spread(df, 'spectral_contrast')
array_column_spread(df_eval, 'spectral_contrast')

### Root Mean Square Energy(RMSE)

In [ ]:
def rmse_feature(df):
    # create an empty list to store the RMSE values
    rmse_list = []

    # iterate through the audio files in the dataset
    for data, rate in zip(df['data'], df['rate']):
        # calculate the root mean square energy
        rmse = librosa.feature.rms(y=data)
        # take the mean of the rmse
        rmse_mean = np.mean(rmse)
        # append the rmse mean to the rmse_list
        rmse_list.append(rmse_mean)

    # add the rmse_list as a new column to the dataframe
    df['rmse'] = rmse_list

In [ ]:
rmse_feature(df)
rmse_feature(df_eval)

### Spectral Flatness (SF)

In [ ]:
def spectral_flatness(df):
    # create an empty list to store the SF values
    sf_list = []

    # iterate through the audio files in the dataset
    for data, rate in zip(df['data'], df['rate']):
        # calculate the spectral flatness
        sf = librosa.feature.spectral_flatness(y=data)
        # take the mean of the SF
        sf_mean = np.mean(sf)
        # append the SF mean to the sf_list
        sf_list.append(sf_mean)

    # add the sf_list as a new column to the dataframe
    df['sf'] = sf_list

In [ ]:
spectral_flatness(df)
spectral_flatness(df_eval)

### Spectral Roll-off (SRO)

In [ ]:
def sro_feature(df):
    # Create an empty list to store the spectral roll-off values
    spectral_rolloff_array = []

    for data, rate in zip(df['data'], df['rate']):
        spectral_rolloff = librosa.feature.spectral_rolloff(y=data, sr=rate)
        spectral_rolloff_mean = np.mean(spectral_rolloff)
        spectral_rolloff_array.append(spectral_rolloff_mean)

    # Add the spectral roll-off values to the dataframe as a new column
    df['spectral_rolloff'] = spectral_rolloff_array

In [ ]:
sro_feature(df)
sro_feature(df_eval)

### Zero-Crossing Rate

In [ ]:
def zcr_feature(df):
    # Create arrays to store the zero-crossing rate values
    zero_crossing_rate_array = []

    for data in df['data']:
        
        # Compute the zero-crossing rate for the current audio file
        zero_crossing_rate = sum(librosa.zero_crossings(data))
        # Append the zero-crossing rate to the zero_crossing_rate_array
        zero_crossing_rate_array.append(zero_crossing_rate)

    # Add the zero-crossing rate arrays as new columns in the dataframe
    df['zero_crossing_rate'] = zero_crossing_rate_array

In [ ]:
zcr_feature(df)
zcr_feature(df_eval)

### Mel-frequency cepstral coefficients (MFCC)

In [ ]:
def mfcc_feature():
    # Create arrays to store the mfcc rate values
    mfcc_array = []

    for data, rate in zip(df['data'], df['rate']):
        
        # Compute the mfccs for the current audio file
        mfcc = librosa.feature.mfcc(y=data, sr=rate, n_mfcc=50)
        # Compute the mean of the mfccs and append it to the mfcc_array
        mfcc_mean = np.mean(mfcc, axis=1)
        mfcc_array.append(mfcc_mean)

    # Add the mfcc as a new column in the dataframe
    df['mfcc'] = mfcc_array

In [ ]:
mfcc_feature(df)
mfcc_feature(df_eval)

In [ ]:
# df['mean_mfcc'] = df['mfcc'].apply(lambda x: sum(x) / len(x))
df['mean_mfcc'] = array_column_spread(df,'mfcc')
df['mean_mfcc'] = array_column_spread(df_eval,'mfcc')


In [ ]:
df.columns

In [ ]:
features = ['Self-reported fluency level ',
       'First Language spoken','speakerId', 'gender', 'ageRange','mean_chroma','mean_tonnetz','mean_spectral_contrast', 
       'rmse', 'sf', 'spectral_rolloff',
       'zero_crossing_rate','mean_mfcc']

In [ ]:
X_eval = df_eval[features].copy()
X_eval

In [ ]:
X = df[features].copy()
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_pred_eval = svm_model(X, y, X_eval)

predictions = pd.DataFrame(y_pred_eval,columns=["Predicted"])
predictions.to_csv('my_data.csv', index=True)

In [ ]:
y_pred = svm_model(X_train, y_train, X_test)
svm_accuracy = accuracy_calculator(y_test, y_pred)

In [ ]:
svm_accuracy

In [ ]:
y_pred_eval = random_forest_model(X , X_eval, y)

predictions = pd.DataFrame(y_pred_eval,columns=["Predicted"])
predictions.to_csv('predictions.csv', index=True)


In [ ]:
y_pred = random_forest_model(X_train, X_test, y_train)
random_forest_accuracy = accuracy_calculator(y_test, y_pred)
random_forest_accuracy

In [ ]:
## Implementation using k-fold
from sklearn.model_selection import KFold, cross_val_score
# define the number of folds and whether to shuffle the data
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# create an instance of the SVM model
clf = SVC()

# use cross_val_score function to perform k-fold cross-validation
scores = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')

# print the mean accuracy and standard deviation
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=0)

# shuffle the data before performing k-fold cross validation
X, y = shuffle(X, y)

# perform k-fold cross validation with 5 folds
scores = cross_val_score(clf, X, y, cv=5)

# calculate the mean accuracy of the model across all folds
accuracy = np.mean(scores)
print("Accuracy:", accuracy)
